# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion

from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///project.db')
df = pd.read_sql_table('project', con = engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
# define feature and target variables X and Y
X = df['message']
Y = df.iloc[:, 4:]

In [4]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [5]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
def display_columns_info(Y):
    for col in Y:
        print(col,': ', Y[col].unique())

In [7]:
display_columns_info(Y)

related :  [1 0 2]
request :  [0 1]
offer :  [0 1]
aid_related :  [0 1]
medical_help :  [0 1]
medical_products :  [0 1]
search_and_rescue :  [0 1]
security :  [0 1]
military :  [0 1]
child_alone :  [0]
water :  [0 1]
food :  [0 1]
shelter :  [0 1]
clothing :  [0 1]
money :  [0 1]
missing_people :  [0 1]
refugees :  [0 1]
death :  [0 1]
other_aid :  [0 1]
infrastructure_related :  [0 1]
transport :  [0 1]
buildings :  [0 1]
electricity :  [0 1]
tools :  [0 1]
hospitals :  [0 1]
shops :  [0 1]
aid_centers :  [0 1]
other_infrastructure :  [0 1]
weather_related :  [0 1]
floods :  [0 1]
storm :  [0 1]
fire :  [0 1]
earthquake :  [0 1]
cold :  [0 1]
other_weather :  [0 1]
direct_report :  [0 1]


In [8]:
# related column
Y['related'].value_counts()

1    19906
0     6122
2      188
Name: related, dtype: int64

In [9]:
# fix the value 2 to 1 (avoid multilabel)
Y['related'][Y['related']  == 2] = 1

In [10]:
# drop child_alone (one 1 label)
Y.drop(columns = ['child_alone'], inplace = True)

In [11]:
display_columns_info(Y)

related :  [1 0]
request :  [0 1]
offer :  [0 1]
aid_related :  [0 1]
medical_help :  [0 1]
medical_products :  [0 1]
search_and_rescue :  [0 1]
security :  [0 1]
military :  [0 1]
water :  [0 1]
food :  [0 1]
shelter :  [0 1]
clothing :  [0 1]
money :  [0 1]
missing_people :  [0 1]
refugees :  [0 1]
death :  [0 1]
other_aid :  [0 1]
infrastructure_related :  [0 1]
transport :  [0 1]
buildings :  [0 1]
electricity :  [0 1]
tools :  [0 1]
hospitals :  [0 1]
shops :  [0 1]
aid_centers :  [0 1]
other_infrastructure :  [0 1]
weather_related :  [0 1]
floods :  [0 1]
storm :  [0 1]
fire :  [0 1]
earthquake :  [0 1]
cold :  [0 1]
other_weather :  [0 1]
direct_report :  [0 1]


### 2. Write a tokenization function to process your text data

In [12]:
def tokenize(text):
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
        
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [13]:
pipeline =  pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(LinearSVC() ,n_jobs=-1)),
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [14]:
# split data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 99)

In [15]:
model = pipeline
model.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...i_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
           n_jobs=-1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [16]:
Y_pred = model.predict(X_test)

In [17]:
Y_pred[:,1]

array([0, 0, 0, ..., 0, 0, 0])

In [18]:
def display_result(Y_test, Y_pred):
    for idx,category in enumerate(list(Y_test.columns.values)):
        test = np.array(Y_test)[:,idx]
        pred = Y_pred[:,idx]
        print('Category: ', category)
        print(classification_report(test, pred))
        print('---------------------------------')

In [19]:
display_result(Y_test, Y_pred)

Category:  related
             precision    recall  f1-score   support

          0       0.68      0.52      0.59      1543
          1       0.86      0.92      0.89      5011

avg / total       0.82      0.83      0.82      6554

---------------------------------
Category:  request
             precision    recall  f1-score   support

          0       0.92      0.96      0.94      5416
          1       0.76      0.59      0.67      1138

avg / total       0.89      0.90      0.89      6554

---------------------------------
Category:  offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6526
          1       0.00      0.00      0.00        28

avg / total       0.99      1.00      0.99      6554

---------------------------------
Category:  aid_related
             precision    recall  f1-score   support

          0       0.78      0.80      0.79      3806
          1       0.72      0.69      0.70      2748

avg / total   

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [20]:
# from sklearn.metrics import accuracy_score, precision_score, recall_score, make_scorer

In [21]:
parameters = {
        'tfidf__use_idf': (True, False),
        'vect__ngram_range': ((1, 1), (1, 2)),
        'vect__max_df': (0.5, 1.0),
#         'clf__n_estimators': [50, 100, 200],
#         'clf__min_samples_split': [2, 3, 4],
#         'features__transformer_weights': (
#             {'text_pipeline': 1, 'starting_verb': 0.5},
#             {'text_pipeline': 0.5, 'starting_verb': 1},
#             {'text_pipeline': 0.8, 'starting_verb': 1},
#         )
}

# accuracy_scorer = make_scorer(accuracy_score)
# precision_scorer = make_scorer(precision_score, average = 'macro')
# recall_scorer = make_scorer(recall_score, average = 'macro')

cv_model = GridSearchCV(pipeline, 
                        param_grid = parameters, 
                        scoring = {
                            'f1_weighted'
                        },
                        refit = 'f1_weighted',
                        return_train_score=True,
                        n_jobs = -1)

In [22]:
cv_model

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...i_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
           n_jobs=-1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'tfidf__use_idf': (True, False), 'vect__ngram_range': ((1, 1), (1, 2)), 'vect__max_df': (0.5, 1.0)},
       pre_dispatch='2*n_jobs', refit='f1_weighted',
       return_train_score=True, scoring={'f1_weighted'}, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [23]:
cv_model.fit(X_train, Y_train)

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:113

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...i_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
           n_jobs=-1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'tfidf__use_idf': (True, False), 'vect__ngram_range': ((1, 1), (1, 2)), 'vect__max_df': (0.5, 1.0)},
       pre_dispatch='2*n_jobs', refit='f1_weighted',
       return_train_score=True, scoring={'f1_weighted'}, verbose=0)

In [24]:
Y_pred_cv = cv_model.predict(X_test)

In [25]:
display_result(Y_test, Y_pred_cv)

Category:  related
             precision    recall  f1-score   support

          0       0.68      0.52      0.59      1543
          1       0.86      0.92      0.89      5011

avg / total       0.82      0.83      0.82      6554

---------------------------------
Category:  request
             precision    recall  f1-score   support

          0       0.92      0.96      0.94      5416
          1       0.76      0.59      0.67      1138

avg / total       0.89      0.90      0.89      6554

---------------------------------
Category:  offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6526
          1       0.00      0.00      0.00        28

avg / total       0.99      1.00      0.99      6554

---------------------------------
Category:  aid_related
             precision    recall  f1-score   support

          0       0.78      0.80      0.79      3806
          1       0.72      0.69      0.70      2748

avg / total   

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [26]:
# use StartingVerbExtractor
from sklearn.base import BaseEstimator, TransformerMixin

class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return 1
        return 0

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [27]:
def build_new_model():
    pipeline = Pipeline([
        ('features', FeatureUnion([
            
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),
            
            ('starting_verb', StartingVerbExtractor()),
        ])),

       ('clf', MultiOutputClassifier(LinearSVC() ,n_jobs=-1)),
    ])
    

    parameters = {
        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
#         'features__text_pipeline__vect__max_features': (None, 5000, 10000),
#         'features__text_pipeline__tfidf__use_idf': (True, False),
#         'clf__n_estimators': [50, 100, 200],
#         'clf__min_samples_split': [2, 3, 4],
        'features__transformer_weights': (
            {'text_pipeline': 1, 'starting_verb': 0.5},
            {'text_pipeline': 0.5, 'starting_verb': 1},
            {'text_pipeline': 0.8, 'starting_verb': 1},
        )
    }

    cv = GridSearchCV(pipeline, 
                      param_grid = parameters, 
                      scoring = {'f1_weighted'},
                      refit = 'f1_weighted',
                      return_train_score=True,
                      n_jobs = -1)
    return cv

In [28]:
new_model = build_new_model()

In [29]:
new_model.fit(X_train, Y_train)

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:113

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:113

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...i_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
           n_jobs=-1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)), 'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0), 'features__transformer_weights': ({'text_pipeline': 1, 'starting_verb': 0.5}, {'text_pipeline': 0.5, 'starting_verb': 1}, {'text_pipeline': 0.8, 'starting_verb': 1})},
       pre_dispatch='2*n_jobs', refit='f1_weighted',
       return_train_score=True, scoring={'f1_weighted'}, verbose=0)

In [30]:
Y_pred_new = new_model.predict(X_test)

In [31]:
display_result(Y_test, Y_pred_new)

Category:  related
             precision    recall  f1-score   support

          0       0.68      0.52      0.59      1543
          1       0.86      0.92      0.89      5011

avg / total       0.82      0.83      0.82      6554

---------------------------------
Category:  request
             precision    recall  f1-score   support

          0       0.92      0.96      0.94      5416
          1       0.76      0.60      0.67      1138

avg / total       0.89      0.90      0.89      6554

---------------------------------
Category:  offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6526
          1       0.00      0.00      0.00        28

avg / total       0.99      1.00      0.99      6554

---------------------------------
Category:  aid_related
             precision    recall  f1-score   support

          0       0.78      0.80      0.79      3806
          1       0.72      0.69      0.70      2748

avg / total   

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [32]:
import pickle
with open('new_model.pickle', 'wb') as f:
    pickle.dump(new_model, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.